In [1]:
import requests
import codecs
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import ast
import urllib2
import time

In [14]:
text = requests.get("http://www.trulia.com/for_rent/New_York,NY/5_p").text
f = open("trulia_part2.txt", 'w')
text = text.encode('utf-8')
f.write(text)
f.close()

In [15]:
text = codecs.open("trulia_part2.txt", 'r','utf-8')
text = BeautifulSoup(text, "html.parser")

In [153]:
text = codecs.open("trulia_part2.txt", 'r','utf-8')
text = BeautifulSoup(text, "html.parser")

result_lst = text.find_all('li', { 'class': 'hover propertyCard property-data-elem clickable' })

list4 = []

for i, apt in enumerate(result_lst):
    dictTable = {}
    
    LatitudeTags  = apt.find_all('meta', {'itemprop': 'latitude'})
    LongitudeTags  = apt.find_all('meta', {'itemprop': 'longitude'})
    dictTable["latitude"] = [tag.get('content').encode('utf-8') for tag in LatitudeTags][0]
    dictTable["longitude"] = [tag.get('content').encode('utf-8') for tag in LongitudeTags][0]
    
    address = result_lst[i].find('div', {'itemprop': "address"}).get_text().encode('utf-8').strip()
    address = filter(bool, map(lambda x: x.strip(), address.split('\n')))
    dictTable["AptName"] = address[0]
    dictTable["Street"] = address[0].split("#")[0]
    dictTable["City"] = address[1]
    dictTable["State"] = address[2]
    dictTable["Postal"] = address[3]
    
    RentRate = result_lst[i].find('div', {'class': "col lastCol txtR"}).get_text().encode('utf-8').strip()
    dictTable["RentRate"] = RentRate[1:].replace(",", "")    
    
    block  = apt.find('div', {'class': 'line simpleTruncate mtm smallLineHeight'})
    roomtype =block.find('div', {'class': 'col cols3'}).get_text().encode('utf-8').strip()
    roomtype = filter(bool, map(lambda x: x.strip(), roomtype.split('\n'))) 
    for i in range(len(roomtype)):
        if re.search('(bd)$', roomtype[i]) != None:
            dictTable['NoOfBedroom'] = roomtype[i]
        if re.search('(ba)$', roomtype[i]) != None:
            dictTable['NoOfBathRoom'] = roomtype[i]    

    AreaRentalInfo =block.find('div', {'class': 'cols4'}).text.encode('utf-8').strip()
    AreaRentalInfo = filter(bool, map(lambda x: x.strip(), AreaRentalInfo.split('\n')))
    
    for i in range(len(AreaRentalInfo)):
        if re.search('(sqft)$', AreaRentalInfo[i]) != None:
            dictTable['Area'] = AreaRentalInfo[i]
        else:
            dictTable['RentalType'] = AreaRentalInfo[i]             
    
    list4.append(dictTable)
df_type = pd.DataFrame(list4)
# df_type



In [125]:
result_lst = text.find_all('li', { 'class': 'hover propertyCard property-data-elem clickable' })

list4 = []
for i, apt in enumerate(result_lst):
    dictTable2 = {}
    address = result_lst[i].find('div', {'itemprop': "address"}).get_text().encode('utf-8').strip()
    address = filter(bool, map(lambda x: x.strip(), address.split('\n')))    
    dictTable2['Aptname'] = address[0]   
    
    block  = apt.find('div', {'class': 'line simpleTruncate mtm smallLineHeight'})
    roomtype =block.find('div', {'class': 'col cols3'}).get_text().encode('utf-8').strip()
    roomtype = filter(bool, map(lambda x: x.strip(), roomtype.split('\n'))) 
    for i in range(len(roomtype)):
        if re.search('(bd)$', roomtype[i]) != None:
            dictTable2['NoOfBedroom'] = roomtype[i]
        if re.search('(ba)$', roomtype[i]) != None:
            dictTable2['NoOfBathRoom'] = roomtype[i]    

    AreaRentalInfo =block.find('div', {'class': 'cols4'}).text.encode('utf-8').strip()
    AreaRentalInfo = filter(bool, map(lambda x: x.strip(), AreaRentalInfo.split('\n')))
    
    for i in range(len(AreaRentalInfo)):
        if re.search('(sqft)$', AreaRentalInfo[i]) != None:
            dictTable2['Area'] = AreaRentalInfo[i]
        else:
            dictTable2['RentalType'] = AreaRentalInfo[i]             
    
    list4.append(dictTable2)
df_type = pd.DataFrame(list4)
# df_type

    
    


In [144]:
import time
text = codecs.open("trulia_part2.txt", 'r','utf-8')
text = BeautifulSoup(text, "html.parser")

result_lst = text.find_all('li', { 'class': 'hover propertyCard property-data-elem clickable' })

list2 = []
for i, apt in enumerate(result_lst):
    dictTable2 = {}
    address = result_lst[i].find('div', {'itemprop': "address"}).get_text().encode('utf-8').strip()
    address = filter(bool, map(lambda x: x.strip(), address.split('\n')))    
    dictTable2['Aptname'] = address[0]      
    
    anchors = result_lst[i].find('a', {'class': "primaryLink pdpLink activeLink"})['href']
    site = "http://www.trulia.com/" + anchors
    response = requests.get(site).text
    text = BeautifulSoup(response, "html.parser")    
    result_lst3 = text.find_all('div', { 'id': 'schoolsAtAGlance' })
    for result in result_lst3:
        body1 = result.find_all('div', { 'class': 'boxBody mvs' })[0].get_text().encode('utf-8').strip()      
        body1 = filter(bool, map(lambda x: x.strip(), body1.split('\n')))

        for i in range(len(body1)):
            if (body1[i] == 'Elementary'):
                dictTable2['ElementarySchool_rating']= body1[i + 1]
            if (body1[i] == 'Middle'):
                dictTable2['MiddleSchool_rating']= body1[i + 1]
            if (body1[i] == 'High'):
                dictTable2['HighSchool_rating']= body1[i + 1] 
    result_lst4 = text.find_all('div', { 'data-action': 'showCrimePopup' })    
    for result in result_lst4:
    #         Crime = {"CrimeLevel": None}
        body2 = result.find('div', { 'class': 'boxBody media pvs h5' }).get_text().encode('utf-8').strip()
        body2 = filter(bool, map(lambda x: x.strip(), body2.split('\n')))      
        dictTable2["CrimeLevel"] = body2[0]
    #         list3.append(Crime)

    list2.append(dictTable2)
df_School_Crime = pd.DataFrame(list2)
# df_School_Crime



In [45]:
def SchoolCrimeInfo(page_url):
    page = requests.get(page_url).text
    text = BeautifulSoup(page, "html.parser")
    
    result_lst = text.find_all('li', { 'class': 'hover propertyCard property-data-elem clickable' })

    list2 = []
    for i, apt in enumerate(result_lst):
        dictTable2 = {}
        address = result_lst[i].find('div', {'itemprop': "address"}).get_text().encode('utf-8').strip()
        address = filter(bool, map(lambda x: x.strip(), address.split('\n')))    
        dictTable2['Aptname'] = address[0]      

        anchors = result_lst[i].find('a', {'class': "primaryLink pdpLink activeLink"})['href']
        site = "http://www.trulia.com/" + anchors
        response = requests.get(site).text
        text = BeautifulSoup(response, "html.parser")    
        result_lst3 = text.find_all('div', { 'id': 'schoolsAtAGlance' })
        for result in result_lst3:
            body1 = result.find_all('div', { 'class': 'boxBody mvs' })[0].get_text().encode('utf-8').strip()      
            body1 = filter(bool, map(lambda x: x.strip(), body1.split('\n')))

            for i in range(len(body1)):
                if (body1[i] == 'Elementary'):
                    dictTable2['ElementarySchool_rating']= body1[i + 1]
                if (body1[i] == 'Middle'):
                    dictTable2['MiddleSchool_rating']= body1[i + 1]
                if (body1[i] == 'High'):
                    dictTable2['HighSchool_rating']= body1[i + 1] 
        result_lst4 = text.find_all('div', { 'data-action': 'showCrimePopup' })    
        for result in result_lst4:
        #         Crime = {"CrimeLevel": None}
            body2 = result.find('div', { 'class': 'boxBody media pvs h5' }).get_text().encode('utf-8').strip()
            body2 = filter(bool, map(lambda x: x.strip(), body2.split('\n')))      
            dictTable2["CrimeLevel"] = body2[0]
        #         list3.append(Crime)

        list2.append(dictTable2)
    df_School_Crime = pd.DataFrame(list2)
    return df_School_Crime



In [ ]:
url = "http://www.trulia.com/for_rent/New_York,NY/5_p"
df_SchoolCrimeInfo = SchoolCrimeInfo(url)
# df_SchoolCrimeInfo

In [46]:
def AptInfo(page_url):
    page = requests.get(page_url).text
    text = BeautifulSoup(page, "html.parser")
    
    result_lst = text.find_all('li', { 'class': 'hover propertyCard property-data-elem clickable' })
    
    list4 = []

    for i, apt in enumerate(result_lst):
        dictTable = {}

        LatitudeTags  = apt.find_all('meta', {'itemprop': 'latitude'})
        LongitudeTags  = apt.find_all('meta', {'itemprop': 'longitude'})
        dictTable["latitude"] = [tag.get('content').encode('utf-8') for tag in LatitudeTags][0]
        dictTable["longitude"] = [tag.get('content').encode('utf-8') for tag in LongitudeTags][0]

        address = result_lst[i].find('div', {'itemprop': "address"}).get_text().encode('utf-8').strip()
        address = filter(bool, map(lambda x: x.strip(), address.split('\n')))
        dictTable["Aptname"] = address[0]
        dictTable["Street"] = address[0].split("#")[0]
        dictTable["City"] = address[1]
        dictTable["State"] = address[2]
        dictTable["Postal"] = address[3]

        RentRate = result_lst[i].find('div', {'class': "col lastCol txtR"}).get_text().encode('utf-8').strip()
        dictTable["RentRate"] = RentRate[1:].replace(",", "")    

        block  = apt.find('div', {'class': 'line simpleTruncate mtm smallLineHeight'})
        roomtype =block.find('div', {'class': 'col cols3'}).get_text().encode('utf-8').strip()
        roomtype = filter(bool, map(lambda x: x.strip(), roomtype.split('\n'))) 
        for i in range(len(roomtype)):
            if re.search('(bd)$', roomtype[i]) != None:
                dictTable['NoOfBedroom'] = roomtype[i]
            if re.search('(ba)$', roomtype[i]) != None:
                dictTable['NoOfBathRoom'] = roomtype[i]    

        AreaRentalInfo =block.find('div', {'class': 'cols4'}).text.encode('utf-8').strip()
        AreaRentalInfo = filter(bool, map(lambda x: x.strip(), AreaRentalInfo.split('\n')))

        for i in range(len(AreaRentalInfo)):
            if re.search('(sqft)$', AreaRentalInfo[i]) != None:
                dictTable['Area'] = AreaRentalInfo[i]
            else:
                dictTable['RentalType'] = AreaRentalInfo[i]             

        list4.append(dictTable)
    df_type = pd.DataFrame(list4)
    return df_type



In [156]:
url = "http://www.trulia.com/for_rent/New_York,NY/5_p"
df_AptInfo = AptInfo(url)
# df_AptInfo

In [53]:
import time
DF_all = pd.DataFrame()
for i in range(250,317):
    urls = "http://www.trulia.com/for_rent/New_York,NY/" + str(i) + "_p"
    print urls
    Df_Temp = AptInfo(urls)
    DF_all = pd.concat([DF_all, Df_Temp])
    time.sleep(1)
DF_all
DF_all.to_excel("trulia_22_AptInfo_250_317.xls")




http://www.trulia.com/for_rent/New_York,NY/250_p
http://www.trulia.com/for_rent/New_York,NY/251_p
http://www.trulia.com/for_rent/New_York,NY/252_p
http://www.trulia.com/for_rent/New_York,NY/253_p
http://www.trulia.com/for_rent/New_York,NY/254_p
http://www.trulia.com/for_rent/New_York,NY/255_p
http://www.trulia.com/for_rent/New_York,NY/256_p
http://www.trulia.com/for_rent/New_York,NY/257_p
http://www.trulia.com/for_rent/New_York,NY/258_p
http://www.trulia.com/for_rent/New_York,NY/259_p
http://www.trulia.com/for_rent/New_York,NY/260_p
http://www.trulia.com/for_rent/New_York,NY/261_p
http://www.trulia.com/for_rent/New_York,NY/262_p
http://www.trulia.com/for_rent/New_York,NY/263_p
http://www.trulia.com/for_rent/New_York,NY/264_p
http://www.trulia.com/for_rent/New_York,NY/265_p
http://www.trulia.com/for_rent/New_York,NY/266_p
http://www.trulia.com/for_rent/New_York,NY/267_p
http://www.trulia.com/for_rent/New_York,NY/268_p
http://www.trulia.com/for_rent/New_York,NY/269_p
http://www.trulia.co

In [33]:
#school Crime takes lot of time
import time
DF_all = pd.DataFrame()
for i in range(300,317):
    urls = "http://www.trulia.com/for_rent/New_York,NY/" + str(i) + "_p"
    print urls
    Df_Temp = SchoolCrimeInfo(urls)
    DF_all = pd.concat([DF_all, Df_Temp])
    time.sleep(1)
DF_all
DF_all.to_excel("trulia_2_SchoolCrime_300_317.xls")

http://www.trulia.com/for_rent/New_York,NY/300_p
http://www.trulia.com/for_rent/New_York,NY/301_p
http://www.trulia.com/for_rent/New_York,NY/302_p
http://www.trulia.com/for_rent/New_York,NY/303_p
http://www.trulia.com/for_rent/New_York,NY/304_p
http://www.trulia.com/for_rent/New_York,NY/305_p
http://www.trulia.com/for_rent/New_York,NY/306_p
http://www.trulia.com/for_rent/New_York,NY/307_p
http://www.trulia.com/for_rent/New_York,NY/308_p
http://www.trulia.com/for_rent/New_York,NY/309_p
http://www.trulia.com/for_rent/New_York,NY/310_p
http://www.trulia.com/for_rent/New_York,NY/311_p
http://www.trulia.com/for_rent/New_York,NY/312_p
http://www.trulia.com/for_rent/New_York,NY/313_p
http://www.trulia.com/for_rent/New_York,NY/314_p
http://www.trulia.com/for_rent/New_York,NY/315_p
http://www.trulia.com/for_rent/New_York,NY/316_p
